In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24186
2,Huelva,Confirmados PDIA 14 días,1645
3,Huelva,Tasa PDIA 14 días,"320,55654071750104"
4,Huelva,Confirmados PDIA 7 días,508
5,Huelva,Total Confirmados,24416
6,Huelva,Curados,13537
7,Huelva,Fallecidos,262


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24186.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6454.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 851 personas en los últimos 7 días 

Un positivo PCR cada 318 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24186.0,508.0,1645.0,513170.0,98.992537,320.556541,68.0
Huelva-Costa,14751.0,332.0,1051.0,289548.0,114.661472,362.979541,54.0
Huelva (capital),6454.0,169.0,453.0,143837.0,117.494108,314.939828,30.0
Cartaya,900.0,37.0,64.0,20083.0,184.235423,318.677488,14.0
Condado-Campiña,6685.0,141.0,469.0,156231.0,90.250975,300.196504,11.0
Almonte,810.0,43.0,99.0,24507.0,175.460073,403.966214,6.0
Isla Cristina,1979.0,28.0,137.0,21393.0,130.883934,640.396391,5.0
Lepe,1957.0,52.0,136.0,27880.0,186.513630,487.804878,4.0
Aljaraque,921.0,13.0,51.0,21474.0,60.538325,237.496507,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,163.0,2.0,30.0,2885.0,69.324090,1039.861352,0.0
Isla Cristina,1979.0,28.0,137.0,21393.0,130.883934,640.396391,5.0
Cumbres de San Bartolomé,5.0,2.0,2.0,376.0,531.914894,531.914894,0.0
Villanueva de los Castillejos,145.0,5.0,15.0,2825.0,176.991150,530.973451,-1.0
Lepe,1957.0,52.0,136.0,27880.0,186.513630,487.804878,4.0
Beas,158.0,2.0,20.0,4341.0,46.072334,460.723336,0.0
Ayamonte,1173.0,15.0,96.0,21104.0,71.076573,454.890068,-2.0
Bollullos Par del Condado,576.0,20.0,63.0,14387.0,139.014388,437.895322,-1.0
Rociana del Condado,300.0,12.0,34.0,7939.0,151.152538,428.265525,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villablanca,163.0,2.0,30.0,2885.0,69.324090,1039.861352,0.0,0.066667
Beas,158.0,2.0,20.0,4341.0,46.072334,460.723336,0.0,0.100000
Puebla de Guzmán,81.0,1.0,10.0,3092.0,32.341527,323.415265,0.0,0.100000
Bonares,178.0,2.0,18.0,6060.0,33.003300,297.029703,-1.0,0.111111
Villalba del Alcor,437.0,2.0,14.0,3366.0,59.417706,415.923945,0.0,0.142857
Ayamonte,1173.0,15.0,96.0,21104.0,71.076573,454.890068,-2.0,0.156250
Almonaster la Real,54.0,1.0,6.0,1788.0,55.928412,335.570470,0.0,0.166667
Trigueros,223.0,3.0,17.0,7862.0,38.158229,216.229967,0.0,0.176471
Palos de la Frontera,438.0,7.0,39.0,11742.0,59.615057,332.141032,0.0,0.179487
